In [5]:
import csv
from datetime import datetime

# Step 1: Extract Data

def read_csv(file_path):
  with open(file_path, mode='r') as file:
    reader = csv.DictReader(file)
    data = [row for row in reader]
    return data

# Step 2: Transform Data
def clean_sales_data(sales):
    """Cleans sales data by removing duplicates and handling missing values"""
    seen = set()
    cleaned_data = []
    for sale in sales:
      sale_id = sale['sale_id'] 
      if sale_id not in seen and all(sale.values()):
        seen.add(sale_id)
        cleaned_data.append(sale)
    return cleaned_data

def integrate_data(sales, products, customers):
    """
    Integrates sales data with product and customer information.
    """
    product_lookup = {p['product_id']: p for p in products}
    customer_lookup = {c['customer_id']: c for c in customers}

    integrated_data = []
    for sale in sales:
        product = product_lookup.get(sale['product_id'], {})
        customer = customer_lookup.get(sale['customer_id'], {})

        if product and customer:  # Ensure both mappings exist#
          # Change the date format to '%m/%d/%Y' to match the data
          integrated_data.append({
            "sale_id": sale['sale_id'],
            "date": sale['date'],
            "customer_id": sale['customer_id'],
            "product_id": sale['product_id'],
            "quantity": int(sale["quantity"]),
            "price": float(sale['price']),
            "product_name": product['product_name'],
            "category": product['category'],
            "customer_name": customer['name'],
            "total_amount": int(sale['quantity']) * float(sale['price']),
            "day_of_week": datetime.strptime(sale['date'], '%m/%d/%Y %H:%M').strftime('%A'),
            "month" : datetime.strptime(sale['date'], '%m/%d/%Y %H:%M').strftime('%B'),
            "quarter": (int(datetime.strptime(sale['date'], '%m/%d/%Y %H:%M').month) - 1) // 3 + 1
        })
            
    # Ensure that the `return` statement is properly aligned outside the loop
    return integrated_data

# Step 3: Load Data

def write_csv(file_path, data, headers):
    """Writes a list of dictionaries to a CSV file."""
    with open(r"C:\\Users\\ghode\\Downloads\\ETL PROJECT\\process_sales.csv", mode='w', newline='') as file:
      writer = csv.DictWriter(file, fieldnames=headers)
      writer.writeheader()
      writer.writerows(data)

# Step 4: Analyze Data
def calculate_sales_by_category(sales):
  category_sales = {}
  for sale in sales:
    category = sale['category']
    amount = float(sale['total_amount'])
    category_sales[category] = category_sales.get(category, 0) + amount
  return category_sales


def identify_top_products(sales, n):
    """Identifies the top N products by total sales."""
    product_sales = {}
    for sale in sales:
        product = sale['product_name']
        amount = float(sale['total_amount'])
        product_sales[product] = product_sales.get(product, 0) + amount
        sorted_products = sorted(product_sales.items(), key=lambda x: x[1], reverse=True)
    return sorted_products[:n]

def analyze_sales_trends(sales):
    """
    Analyzes sales trends over time: monthly, daily, and quarterly.

    :param sales: List of dictionaries containing sales data.
    :return: A dictionary containing monthly, daily, and quarterly sales trends.
    """
    monthly_sales = {}
    daily_sales = {}
    quarterly_sales = {}

    # Process each sale
    for sale in sales:
        # Extract relevant fields
        month = sale['month']
        day_of_week = sale['day_of_week']
        quarter = f"Q{sale['quarter']}"
        amount = float(sale['total_amount'])

        # Aggregate sales
        monthly_sales[month] = monthly_sales.get(month, 0) + amount
        daily_sales[day_of_week] = daily_sales.get(day_of_week, 0) + amount
        quarterly_sales[quarter] = quarterly_sales.get(quarter, 0) + amount

    # Return all trends
    return {
        "monthly_sales": monthly_sales,
        "daily_sales": daily_sales,
        "quarterly_sales": quarterly_sales,
    }


def generate_summary_report(analysis):
    """
    Generates a summary report of the analysis.

    :param analysis: A dictionary containing analysis results (sales by category, top products, sales trends).
    :return: A formatted string containing the summary report.
    """
    report_lines = ["Sales Summary Report", "=" * 20]

    # Sales by Category
    if "sales_by_category" in analysis:
        report_lines.append("\nSales by Category:")
        for category, amount in analysis['sales_by_category'].items():
            report_lines.append(f"{category}: ${amount:.2f}")

    # Top Products
    if "top_products" in analysis:
        report_lines.append("\nTop Products:")
        for product, amount in analysis['top_products']:
            report_lines.append(f"{product}: ${amount:.2f}")

    # Monthly Sales Trends
    if "monthly_sales" in analysis:
        report_lines.append("\nMonthly Sales Trends:")
        for month, amount in analysis['monthly_sales'].items():
            report_lines.append(f"{month}: ${amount:.2f}")

    # Daily Sales Trends
    if "daily_sales" in analysis:
        report_lines.append("\nDaily Sales Trends:")
        for day, amount in analysis['daily_sales'].items():
            report_lines.append(f"{day}: ${amount:.2f}")

    # Quarterly Sales Trends
    if "quarterly_sales" in analysis:
        report_lines.append("\nQuarterly Sales Trends:")
        for quarter, amount in analysis['quarterly_sales'].items():
            report_lines.append(f"{quarter}: ${amount:.2f}")

    # Join all lines and return the report
    return "\n".join(report_lines)


# File paths
customer_file = ("C:\\Users\\ghode\\Downloads\\ETL PROJECT\\Customer.csv")
employee_file = (r"C:\\Users\\ghode\Downloads\\ETL PROJECT\\employee.csv")
product_file = ("C:\\Users\\ghode\\Downloads\\ETL PROJECT\\Product.csv")
sales_file = ("C:\\Users\\ghode\\Downloads\\ETL PROJECT\\sales.csv")

# Extract
customers = read_csv(customer_file)
employees = read_csv(employee_file)
products = read_csv(product_file)
sales = read_csv(sales_file)
#print(sales)
#(products)
#print(customers)

# Clean sales data
cleaned_sales=clean_sales_data(sales)
print("cleaned_sales_data:")
for sales in cleaned_sales:
 print(sales)


# Transform data
transformed_sales = integrate_data(cleaned_sales, products, customers)


# write transform data to CSV file
output_headers = list(transformed_sales[0].keys())
write_csv("Downloads/processed_sales.csv", transformed_sales, output_headers)

# Analyze
top_products = identify_top_products(transformed_sales,n=5) # Print first 5 cleaned records
sales_by_category = calculate_sales_by_category(transformed_sales)
trends = analyze_sales_trends(transformed_sales)


# Combine analysis results
analysis = {
    'top_products': top_products,
    'sales_by_category': sales_by_category,
    **trends  # Include monthly, daily, and quarterly sales trends
}

# Generate & write summary report
summary = generate_summary_report(analysis)

with open("Downloads/summary_report.txt", "w") as report_file:
    report_file.write(summary)

# Print summary to console
print(summary)

cleaned_sales_data:
{'sale_id': '1001', 'date': '9/1/2024 0:00', 'customer_id': 'C101', 'product_id': 'P201', 'quantity': '2', 'price': '29.99'}
{'sale_id': '1002', 'date': '9/2/2024 0:00', 'customer_id': 'C102', 'product_id': 'P202', 'quantity': '1', 'price': '49.99'}
{'sale_id': '1003', 'date': '9/2/2024 0:00', 'customer_id': 'C103', 'product_id': 'P203', 'quantity': '3', 'price': '15.99'}
{'sale_id': '1004', 'date': '9/3/2024 0:00', 'customer_id': 'C101', 'product_id': 'P204', 'quantity': '1', 'price': '99.99'}
{'sale_id': '1005', 'date': '9/4/2024 0:00', 'customer_id': 'C104', 'product_id': 'P201', 'quantity': '2', 'price': '29.99'}
{'sale_id': '1006', 'date': '9/5/2024 0:00', 'customer_id': 'C105', 'product_id': 'P202', 'quantity': '4', 'price': '49.99'}
{'sale_id': '1007', 'date': '9/6/2024 0:00', 'customer_id': 'C106', 'product_id': 'P203', 'quantity': '1', 'price': '15.99'}
{'sale_id': '1008', 'date': '9/7/2024 0:00', 'customer_id': 'C107', 'product_id': 'P204', 'quantity': '5'